In [1]:
load_model = 'googlenet_based_model2.pt'

data_dir = '../../../stanford_dogs_new/'

In [2]:
# import numpy as np

from IPython.display import Image

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from torchvision.datasets import ImageFolder
from PIL import Image

from torch.autograd import Variable

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


from IPython.display import Image
from IPython.display import Markdown

from PIL import Image
import PIL.Image

import pandas as pd

# sklearn metrics
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score


## load the Googlenet based model

In [3]:
googlenet_loaded = models.googlenet(pretrained=True)

for param in googlenet_loaded.parameters():
    param.requires_grad = False

googlenet_loaded.fc = nn.Linear(in_features=1024, out_features=120, bias=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
googlenet_loaded = googlenet_loaded.to(device)



googlenet_loaded.load_state_dict(torch.load('googlenet_based_model2.pt'))
googlenet_loaded.eval()

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /home/rka73/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [4]:
model = googlenet_loaded

In [5]:
# preprocess the image for the model
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
])

In [6]:
path = '../../../stanford_dogs_new/test'

list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

# gets the breed name from subfolders name like 'n02099429-curly-coated_retriever'
def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

per_class_accuracy = dict()
total_accuracy = 0

y_true = []
y_pred = []

for breed_dir in list_subfolders_with_paths:
    
    breed = rename(breed_dir.split('/')[-1])
    
    images = os.listdir(breed_dir)
    
    matched = False
    matches = 0
    total_images = len(images)

    for image in images:

        input_image = Image.open(breed_dir + '/' + image)


        input_tensor = preprocess(input_image)
        if torch.cuda.is_available():
            input_tensor = Variable(input_tensor.cuda())

        input_batch = input_tensor.unsqueeze(0)
        out = model(input_batch)

        probabilities = torch.nn.functional.softmax(out[0], dim=0)
        # print(probabilities)

        with open("../../stanford_dogs_breeds_classes_final.txt", "r") as f:
            categories = [s.strip() for s in f.readlines()]

        predicted_breeds = []
        top1_prob, top1_catid = torch.topk(probabilities, 1)
        for i in range(top1_prob.size(0)):
            # predicted_breeds.append([categories[top3_catid[i]], top3_prob[i].item()*100])
            predicted_breed = categories[top1_catid[i]]
            y_true.append(breed)
            y_pred.append(predicted_breed)

        # list to be used directly by the application (predicted_breed, probability)
        # print("predicted_breeds are: \n", predicted_breed)
        if breed == predicted_breed:
            matched = True
            matches = matches + 1

        per_class_accuracy[breed] = matches / total_images * 100

/home/rka73/.local/lib/python3.8/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


In [7]:
df = pd.DataFrame(list(zip(y_true, y_pred)),
               columns =['Actual_breed', 'Predicted_breed'])

df

,Actual_breed,Predicted_breed
0,Pomeranian,Pomeranian
1,Pomeranian,Pomeranian
2,Pomeranian,Pomeranian
3,Pomeranian,Pomeranian
4,Pomeranian,Pomeranian
...,...,...
2105,papillon,English setter
2106,papillon,papillon
2107,papillon,papillon
2108,papillon,papillon


## Confusion Matrix

In [8]:
# Print the confusion matrix
print(metrics.confusion_matrix(y_true, y_pred))


[[23  0  0 ...  0  0  0]
 [ 0 17  0 ...  0  0  0]
 [ 0  0 21 ...  0  0  0]
 ...
 [ 0  0  0 ... 12  0  0]
 [ 0  1  0 ...  0  8  0]
 [ 0  1  1 ...  0  0 10]]


## Precision, Recall and F1

In [9]:
# Print the precision and recall, among other metrics
print(metrics.classification_report(y_true, y_pred, digits=3))

                                precision    recall  f1-score   support

                  Afghan hound      0.958     0.958     0.958        24
           African hunting dog      0.773     1.000     0.872        17
                      Airedale      0.656     1.000     0.792        21
American Staffordshire terrier      0.684     0.765     0.722        17
                   Appenzeller      0.800     0.500     0.615        16
            Australian terrier      0.824     0.700     0.757        20
            Bedlington terrier      0.826     1.000     0.905        19
          Bernese mountain dog      0.733     1.000     0.846        22
              Blenheim spaniel      0.864     1.000     0.927        19
                 Border collie      0.579     0.733     0.647        15
                Border terrier      0.810     0.944     0.872        18
                   Boston bull      0.739     0.895     0.810        19
          Bouvier des Flandres      0.727     0.533     0.615  

## Macro F1 Score

In [10]:
from sklearn.metrics import f1_score

f1_score(y_true, y_pred, average='macro')


0.7676614982945684

## Micro F1 Score

In [11]:
f1_score(y_true, y_pred, average='micro')


0.7777251184834123

## Weighted F1 Score

In [12]:
f1_score(y_true, y_pred, average='weighted')

0.7707646837771984

## Cohen Kappa Score

In [13]:
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(y_true, y_pred)

0.7758079468458288

## Total Accuracy calculated from per class accuracy

In [14]:
# calculated from per class accuracy
total_accuracy = sum(per_class_accuracy.values())/120
total_accuracy

77.17787542265096

## Per Breed Accuracy

### Top 10 Breeds with least accuracy

In [15]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1])[:k]

[('Eskimo dog', 13.333333333333334),
 ('standard schnauzer', 25.0),
 ('miniature poodle', 31.25),
 ('collie', 31.25),
 ('Lakeland terrier', 35.0),
 ('Lhasa', 42.10526315789473),
 ('whippet', 42.10526315789473),
 ('Appenzeller', 50.0),
 ('Walker hound', 50.0),
 ('Bouvier des Flandres', 53.333333333333336)]

### Top 10 Breeds with most accuracy

In [16]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1], reverse=True)[:k]

[('Japanese spaniel', 100.0),
 ('Blenheim spaniel', 100.0),
 ('schipperke', 100.0),
 ('Old English sheepdog', 100.0),
 ('Samoyed', 100.0),
 ('bull mastiff', 100.0),
 ('Rottweiler', 100.0),
 ('Bernese mountain dog', 100.0),
 ('groenendael', 100.0),
 ('African hunting dog', 100.0)]